<a href="https://colab.research.google.com/github/ebpetrov/w209_vis/blob/master/jeopary_json_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# W209 Final Project
## Jeopardy!

Data Ingestion and Munging Notebook


### A. Import Libraries

In [0]:
import io
import os
import pickle
import json
import copy

import pandas as pd
import numpy as np
import pandas as pd
import nltk

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from pandas.io.json import json_normalize # Functions to flatten json
from pprint import pprint

# Suppress new window for plots
%matplotlib inline 

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from sklearn.model_selection import train_test_split

from sklearn.utils.multiclass import unique_labels
from sklearn.utils import shuffle

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve

from random import choice

In [4]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### B. Import JSON Dataset

In [0]:
filename = '/content/drive/My Drive/Colab Notebooks/W209 Final Project/data/all_seasons.json'
data_listofdict = json.load(open(filename, 'r', encoding='utf-8'))

### C. Functions

In [0]:
def download_csv(filename):
  df_clues.to_csv(filename, index=False)
  files.download(filename)
  return

In [0]:
# Functions
def getList_dictkeys(dict):
    return list(dict.keys())

In [0]:
def prt_dict_value_types(dict_name):
    list_keys = getList_dictkeys(dict_name)
    print()
    for i in list_keys:
        print(type(dict_name[i]), "\t", i, sep="")
    print()
    return

In [0]:
def prt_dict_value_types_2(dict_name):
    list_keys = getList_dictkeys(dict_name)
    print()
    for i in list_keys:
        print(type(dict_name[i][0]), "\t", i, sep="")
    print()
    return

In [0]:
# https://hackersandslackers.com/extract-data-from-complex-json-python/
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [0]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [0]:
def flattenColumn(input, column):
    '''column is a string of the column's name.
    for each value of the column's element (which might be a list),
    duplicate the rest of columns at the corresponding row with the (each) value.
    '''
    column_flat = pd.DataFrame([[i, c_flattened] for i, y in input[column].apply(list).iteritems() for c_flattened in y], columns=['I', column])
    column_flat = column_flat.set_index('I')
    return input.drop(column, 1).merge(column_flat, left_index=True, right_index=True)

In [0]:
def releaseDF(df):
    '''
    Release dataframe from memory
    '''
    list_df = [df]
    del list_df
    return

In [0]:
def plt_density_hist(data_element, title_xaxis):
  # Density plot and histogram
  bin_count = 15
  x = data_element
  x_mu = x.mean()
  x_std = x.std()
  chart_title = "Density and Histogram for " + title_xaxis
  chart_xax = title_xaxis
  chart_yax = "Frequency/Density"

  sns.distplot(x, 
              hist=True, 
              kde=True, 
              bins=bin_count, 
              color = 'darkblue', 
              hist_kws={'edgecolor':'black'},
              kde_kws={'linewidth': 4})

  plt.title(chart_title, fontsize=14)
  plt.xlabel(chart_xax, fontsize=12)
  plt.ylabel(chart_yax, fontsize=12)

  x_norm = np.random.normal(loc = x_mu, scale = x_std, size = 5000)
  sns.kdeplot(x_norm, color="yellow", shade = True)

  plt.show()
  return

def list_tokens(df_column):
  tokens = set()
  for i in range(0, len(df_column)):
    token_list = word_tokenize(df_column[i])
    for j in token_list:
      tokens.add(j)
  tokens = list(tokens)
  tokens.sort()
  return tokens

# Check for occurence of substrings
def check_substrings(df_column, substring):
  text_lst = list(df.text_clean)
  exceptions = [i for i in text_lst if substring in i]
  print(*exceptions, sep = "\n")
  return


def compare_raw_clean_text(df_raw, df_clean, substring):
  if df_raw.str.contains(substring).any():
    print(substring, "occurs in the raw text.")
  else:
    print(substring, "does **NOT** occur in the raw text.")

  print()
  if df_clean.str.contains(substring).any():
    print(substring, "occurs in the cleansed text.")
  else:
    print(substring, "does **NOT** occur in the cleansed text.")
  return

def get_normalized_embeddings():
  return F.normalize(lang_mod.model[0].encoder.weight)

def most_similar(token, embs):
  idx = data_lm.vocab.itos.index(token)
  sims = (embs[idx] @ embs.t()).cpu().detach().numpy()

  print(f'Similar to: {token}')
  print("-"*34)
  for sim_idx in np.argsort(sims)[::-1][1:11]:
    print(f'{data_lm.vocab.itos[sim_idx]:<30}{sims[sim_idx]:.02f}')

In [0]:
# Normalize data using regular expressions
def text_normalize(df, text_field):
  df[text_field] = df[text_field].str.lower()
  df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9]", " ")
  return df

In [0]:
# Remove stop words
stop_words = set(stopwords.words('english'))

def text_nostops(df, text_field):
  df[text_field].apply(lambda x: [item for item in x if item not in stop_words])
  return df

In [0]:
# Lemmatize clean text
lemmatizer = WordNetLemmatizer()

def text_lemmatize(df, text_field):
  for i in range(0, len(df[text_field])):
    df[text_field][i] = lemmatize_sentence(df[text_field][i])
  return df

In [0]:
def text_dataprep(df, text_field):
  df = text_normalize(df, text_field)
  df = text_nostops(df, text_field)
  df = text_lemmatize(df, text_field)
  return df

In [0]:
#https://medium.com/@gaurav5430/using-nltk-for-lemmatizing-sentences-c1bfff963258
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [0]:
#https://medium.com/@gaurav5430/using-nltk-for-lemmatizing-sentences-c1bfff963258
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [0]:
def plot_cm(y_true, y_pred, figsize=(8,6)):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    #cm.index.name = 'Actual Label\n'
    #cm.columns.name = '\nPredicted Label'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, 
                cmap= "Blues", 
                annot=annot, annot_kws={"size": 8}, 
                fmt='', ax=ax)
    ax.set_title('Confusion Matrix\n\n', fontsize=14); 
    ax.set_xlabel('\n\nPredicted Label', fontsize=12);
    ax.set_ylabel('Actual Label\n\n', fontsize=12); 
    ax.xaxis.set_ticklabels(tick_labels, rotation=0); 
    ax.yaxis.set_ticklabels(tick_labels, rotation=0);
    return

### D. Munging

#### D.1 Create preliminary master dataframes

In [0]:
col_games = ['SeasonName', 'StartDate', 'EndDate', 'SeasonUrl', 'GameId', 'ShowNumber', 'AirDate']
col_players = ['GameId', 'Players']
col_rounds = ['GameId', 'Rounds']
col_gamestate = ['GameId', 'GameState']

In [0]:
df_all_games = pd.DataFrame(columns=col_games)
df_all_players = pd.DataFrame(columns=col_players)
df_all_rounds = pd.DataFrame(columns=col_rounds)
df_all_gamestate = pd.DataFrame(columns=col_gamestate)

In [0]:
# Create master dataframes (df_all_games, df_all_players, df_all_rounds, df_all_gamestate)

for i in data_listofdict:
    season_dict_i = i
    
    df_i = pd.DataFrame(season_dict_i)
    df_i_master = df_i.drop("Games", axis=1).join(pd.DataFrame(df_i.Games.values.tolist()))
    
    # Clean dataframe
    df_i_games = df_i_master.copy(deep=True)
    df_i_games = df_i_games[col_games]
    df_all_games = df_all_games.append(df_i_games, ignore_index=True)
        
    df_i_players = df_i_master.copy(deep=True)
    df_i_players = df_i_players[col_players]
    df_all_players = df_all_players.append(df_i_players, ignore_index=True)
    
    df_i_rounds = df_i_master.copy(deep=True)
    df_i_rounds = df_i_rounds[col_rounds]
    df_all_rounds = df_all_rounds.append(df_i_rounds, ignore_index=True)
    
    df_i_gamestate = df_i_master.copy(deep=True)
    df_i_gamestate = df_i_gamestate[col_gamestate]
    df_all_gamestate = df_all_gamestate.append(df_i_gamestate, ignore_index=True)

# Release unused dataframes from memory
list_df = [df_i, df_i_games, df_i_players, df_i_rounds, df_i_gamestate]
for i in list_df:
    releaseDF(i)

In [24]:
# Check dataframe shapes
print()
print("Shape:")
print("-"*40)
print("df_all_games\t", df_all_games.shape, sep="\t")
print("df_all_players\t", df_all_players.shape, sep="\t")
print("df_all_gamestate", df_all_gamestate.shape, sep="\t")
print("df_all_rounds\t", df_all_rounds.shape, sep="\t")


Shape:
----------------------------------------
df_all_games		(6557, 7)
df_all_players		(6557, 2)
df_all_gamestate	(6557, 2)
df_all_rounds		(6557, 2)


#### Create games dataframe

In [62]:
df_games = df_all_games
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6557 entries, 0 to 6556
Data columns (total 7 columns):
SeasonName    6557 non-null object
StartDate     6557 non-null object
EndDate       6557 non-null object
SeasonUrl     6557 non-null object
GameId        6557 non-null object
ShowNumber    6557 non-null object
AirDate       6557 non-null object
dtypes: object(7)
memory usage: 358.7+ KB


In [63]:
convert_dict = {'SeasonName': str,
                'StartDate': str,
                'EndDate': str,
                'SeasonUrl': str, 
                'GameId': int,
                'ShowNumber': str, 
                'AirDate': str}
df_games = df_games.astype(convert_dict)
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6557 entries, 0 to 6556
Data columns (total 7 columns):
SeasonName    6557 non-null object
StartDate     6557 non-null object
EndDate       6557 non-null object
SeasonUrl     6557 non-null object
GameId        6557 non-null int64
ShowNumber    6557 non-null object
AirDate       6557 non-null object
dtypes: int64(1), object(6)
memory usage: 358.7+ KB


In [64]:
df_games.head(5)
#download_csv('df_games.csv')

,SeasonName,StartDate,EndDate,SeasonUrl,GameId,ShowNumber,AirDate
0,Jeopardy!: The Greatest of All Time,2020-01-07,2020-01-14,http://www.j-archive.com/showseason.php?season...,6527,The Greatest of All Time game #8,2020-01-14
1,Jeopardy!: The Greatest of All Time,2020-01-07,2020-01-14,http://www.j-archive.com/showseason.php?season...,6526,The Greatest of All Time game #7,2020-01-14
2,Jeopardy!: The Greatest of All Time,2020-01-07,2020-01-14,http://www.j-archive.com/showseason.php?season...,6522,The Greatest of All Time game #6,2020-01-09
3,Jeopardy!: The Greatest of All Time,2020-01-07,2020-01-14,http://www.j-archive.com/showseason.php?season...,6521,The Greatest of All Time game #5,2020-01-09
4,Jeopardy!: The Greatest of All Time,2020-01-07,2020-01-14,http://www.j-archive.com/showseason.php?season...,6519,The Greatest of All Time game #4,2020-01-08


#### Create players dataframe

In [0]:
# Create df_players
df = df_all_players
n_df = df.shape[0]
n_players = 3
columns = ['GameId', 'PlayerId', 'Name', 'NickName', 'Desc']
keys = copy.deepcopy(columns)
keys.pop(0)

df_players = pd.DataFrame(columns=columns)


for i in range(0, n_df):
    gameid = df_all_players.GameId[i]
    obj = df_all_players.Players[i]
    
    list_GameId = [df_all_players.GameId[i]] * n_players
    list_PlayerId = extract_values(obj, keys[0])
    list_Name = extract_values(obj, keys[1])
    list_NickName = extract_values(obj, keys[2])
    list_Desc = extract_values(obj, keys[3])
    
    df_temp = pd.DataFrame(list(zip(list_GameId, 
                                    list_PlayerId, 
                                    list_Name, 
                                    list_NickName, 
                                    list_Desc)), 
                           columns=columns)
    df_players = df_players.append(df_temp, ignore_index=True)

# Release unused dataframes from memory
list_df = [df, df_all_players, df_temp]
for i in list_df:
    releaseDF(i)

In [85]:
convert_dict = {'GameId': int,
                'PlayerId': int,
                'Name': str,
                'NickName': str,
                'Desc': str}
df_players = df_players.astype(convert_dict)
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19584 entries, 0 to 19583
Data columns (total 5 columns):
GameId      19584 non-null int64
PlayerId    19584 non-null int64
Name        19584 non-null object
NickName    19584 non-null object
Desc        19584 non-null object
dtypes: int64(2), object(3)
memory usage: 765.1+ KB


In [132]:
df_players["clean_Desc"] = df_players.Desc
df_players = text_dataprep(df_players, "clean_Desc")
df_players.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,GameId,PlayerId,Name,NickName,Desc,clean_Desc
0,6527,13085,Brad Rutter,Brad,"the biggest money winner from Los Angeles, Cal...",the big money winner from los angeles californ...
1,6527,13086,Ken Jennings,Ken,the winner of 74 consecutive games from Seattl...,the winner of 74 consecutive game from seattle...
2,6527,13087,James Holzhauer,James,the setter of 20 <i>Jeopardy!</i> records from...,the setter of 20 i jeopardy i record from las ...
3,6526,13085,Brad Rutter,Brad,"the biggest money winner from Los Angeles, Cal...",the big money winner from los angeles california
4,6526,13086,Ken Jennings,Ken,the winner of 74 consecutive games from Seattl...,the winner of 74 consecutive game from seattle...




---



In [0]:
#download_csv('df_players.csv')

#### Create game state dataframe

In [28]:
# Create df_gamestate

df = df_all_gamestate
n_df = df.shape[0]
columns = ["GameId", "Amount", "PlayerId", "NickName", "TeamName"]
keys = copy.deepcopy(columns)
keys.pop(0)

df_gamestate = pd.DataFrame(columns=columns)


for i in range(0, n_df):
    df_gameid = pd.DataFrame([df_all_gamestate.GameId[i]]*n_players, columns=["GameId"])
    obj = df_all_gamestate.GameState[i]
    
    df_temp = json_normalize(obj, sep="_")
    df_temp.rename(columns={"Player_PlayerId":"PlayerId", "Player_NickName":"NickName", "Player_TeamName":"TeamName"}, inplace=True)
    df_temp = pd.concat([df_gameid, df_temp], axis=1, sort=True) #Sorting because non-concatenation axis is not aligned
    
    df_gamestate = df_gamestate.append(df_temp, ignore_index=True)

# Release unused dataframes from memory
list_df = [df, df_all_gamestate, df_temp]
for i in list_df:
    releaseDF(i)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,Amount,GameId,NickName,PlayerId,TeamName
0,1400.0,6527,Brad,13085,NaN
1,23000.0,6527,Ken,13086,NaN
2,NaN,6527,James,13087,NaN
3,NaN,6526,Brad,13085,NaN
4,65600.0,6526,Ken,13086,NaN


In [135]:
df_gamestate.head(5)

,Amount,GameId,NickName,PlayerId,TeamName
0,1400.0,6527,Brad,13085,NaN
1,23000.0,6527,Ken,13086,NaN
2,NaN,6527,James,13087,NaN
3,NaN,6526,Brad,13085,NaN
4,65600.0,6526,Ken,13086,NaN


In [0]:
# Leave NaN values for Amount and TeamName
#download_csv('df_gamestate.csv')

#### Create rounds dataframe

In [30]:
# Create df_rounds
df = df_all_rounds
n_df = df.shape[0]
columns = ['GameId', 'Categories', 'Clues', 'GameState', 'HasScoreCorrection', 'RoundType']
keys = copy.deepcopy(columns)
keys.pop(0)

df_rounds = pd.DataFrame(columns=columns)

for i in range(0, n_df):
    gameid = df_all_rounds.GameId[i]
    obj = df_all_rounds.Rounds[i]
    
    df_temp = json_normalize(obj, sep="_")
    df_gameid = pd.DataFrame([gameid] * df_temp.shape[0], columns=["GameId"])
    df_temp = pd.concat([df_gameid, df_temp], axis=1)
    
    df_rounds = df_rounds.append(df_temp, ignore_index=True)

# Release unused dataframes from memory
list_df = [df, df_all_rounds, df_gameid]
for i in list_df:
    releaseDF(i)
    
df_rounds = df_rounds[['GameId', 'RoundType', 'HasScoreCorrection', 'Categories', 'Clues', 'GameState']]

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [31]:
# Replace NaN values in RountType with "First Round"
df_rounds["RoundType"].fillna("First Round", inplace = True) 

# Replace NaN values in HasScoreCorrection with "False"
df_rounds["HasScoreCorrection"].fillna(False, inplace = True) 

df_rounds.head(5)

,GameId,RoundType,HasScoreCorrection,Categories,Clues,GameState
0,6527,First Round,False,"[{'Name': 'BROADWAY', 'Order': 1}, {'Name': 'N...","[{'Order': 1, 'Category': {'Name': 'LITERARY T...","[{'Player': {'PlayerId': 13085, 'NickName': 'B..."
1,6527,Double Jeopardy,False,"[{'Name': 'ONCE UPON A TIME IN HOLLYWOOD', 'Or...","[{'Order': 1, 'Category': {'Name': 'SURPRISE M...","[{'Player': {'PlayerId': 13087, 'NickName': 'J..."
2,6527,Final Jeopardy,False,"[{'Name': 'SHAKESPEARE'S TRAGEDIES', 'Order': 1}]",[{'Category': {'Name': 'SHAKESPEARE'S TRAGEDIE...,"[{'Player': {'PlayerId': 13085, 'NickName': 'B..."
3,6526,First Round,False,"[{'Name': 'MODERN FAMILY', 'Order': 1}, {'Name...","[{'Order': 1, 'Category': {'Name': 'KNIGHT LIN...","[{'Player': {'PlayerId': 13085, 'NickName': 'B..."
4,6526,Double Jeopardy,False,"[{'Name': 'THE WORLD ACCORDING TO KAREEM', 'Or...","[{'Order': 1, 'Category': {'Name': 'HISTORY', ...","[{'Player': {'PlayerId': 13087, 'NickName': 'J..."


#### Create categories dataframe

In [32]:
# Create df_categories

df_categories = df_rounds.copy(deep=True)
df_categories = df_categories[['GameId', 'RoundType', 'HasScoreCorrection', 'Categories']]
df_categories = flattenColumn(df_categories, "Categories")
df_categories.reset_index(inplace=True)
df_categories = df_categories.drop(columns=["index"])
df_categories_norm = json_normalize(df_categories["Categories"])

df_categories_merge = df_categories.merge(df_categories_norm, left_index=True, right_index=True)
df_categories = df_categories_merge.drop(columns=["Categories"])
df_categories.rename(columns={"Name":"CategoryName"}, inplace=True)

# Release unused dataframes from memory
list_df = [df_categories_norm, df_categories_merge]
for i in list_df:
    releaseDF(i)


,GameId,RoundType,HasScoreCorrection,CategoryName,Order
0,6527,First Round,False,BROADWAY,1
1,6527,First Round,False,NONSENSE WORDS,2
2,6527,First Round,False,LITERARY TERMS & DEVICES,3
3,6527,First Round,False,THE GREATEST CANADIANS OF ALL TIME,4
4,6527,First Round,False,MATH TO ROMAN NUMERALS TO INITIALS TO NAMES,5


In [137]:
convert_dict = {'GameId': int,
                'RoundType': str, 
                'HasScoreCorrection': bool,
                'CategoryName': str,
                'Order':int}
df_categories = df_categories.astype(convert_dict)
df_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84760 entries, 0 to 84759
Data columns (total 5 columns):
GameId                84760 non-null int64
RoundType             84760 non-null object
HasScoreCorrection    84760 non-null bool
CategoryName          84760 non-null object
Order                 84760 non-null int64
dtypes: bool(1), int64(2), object(2)
memory usage: 2.7+ MB


In [138]:
df_categories["clean_CategoryName"] = df_categories.CategoryName
df_categories = text_dataprep(df_categories, "clean_CategoryName")
df_categories.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,GameId,RoundType,HasScoreCorrection,CategoryName,Order,clean_CategoryName
0,6527,First Round,False,BROADWAY,1,broadway
1,6527,First Round,False,NONSENSE WORDS,2,nonsense word
2,6527,First Round,False,LITERARY TERMS & DEVICES,3,literary term device
3,6527,First Round,False,THE GREATEST CANADIANS OF ALL TIME,4,the great canadian of all time
4,6527,First Round,False,MATH TO ROMAN NUMERALS TO INITIALS TO NAMES,5,math to roman numeral to initial to name


In [0]:
#download_csv('df_categories.csv')

#### Create categories dataframe

In [34]:
# Create df_gamescore
## THIS SHOULD YIELD THE SAME RESULT AS df_gamestate
## CHECK THIS

df_gamescore = df_rounds.copy(deep=True)
df_gamescore = df_gamescore[['GameId', 'RoundType', 'HasScoreCorrection', 'GameState']]
df_gamescore = flattenColumn(df_gamescore, "GameState")
df_gamescore.reset_index(inplace=True)
df_gamescore = df_gamescore.drop(columns=["index"])
df_gamescore_norm = json_normalize(df_gamescore["GameState"])

df_gamescore_merge = df_gamescore.merge(df_gamescore_norm, left_index=True, right_index=True)
df_gamescore = df_gamescore_merge.drop(columns=["GameState"])
df_gamescore.columns = df_gamescore.columns.map(lambda x: x.split(".")[-1]) # Remove .prefix of column name

# Release unused dataframes from memory
list_df = [df_gamescore_norm, df_gamescore_merge]
for i in list_df:
    releaseDF(i)

df_gamescore.head(5)

,GameId,RoundType,HasScoreCorrection,Amount,PlayerId,NickName,TeamName
0,6527,First Round,False,1000.0,13085,Brad,NaN
1,6527,First Round,False,5600.0,13086,Ken,NaN
2,6527,First Round,False,8200.0,13087,James,NaN
3,6527,Double Jeopardy,False,44000.0,13087,James,NaN
4,6527,Double Jeopardy,False,23000.0,13086,Ken,NaN


In [0]:
# Leave TeamName NaNs as is
#download_csv('df_gamescore.csv')

#### Create clues dataframe

In [0]:
# Create df_clues

df_clues = df_rounds.copy(deep=True)
df_clues = df_clues[['GameId', 'RoundType', 'HasScoreCorrection', 'Clues']]

df_clues = flattenColumn(df_clues, "Clues")
df_clues.reset_index(inplace=True)
df_clues = df_clues.drop(columns=["index"])

df_clues_norm1 = json_normalize(df_clues["Clues"])
df_clues_merge = df_clues.merge(df_clues_norm1, left_index=True, right_index=True)
df_clues = df_clues_merge.drop(columns=["Clues"])

df_clues = flattenColumn(df_clues, "PlayerAnswers")
df_clues.reset_index(inplace=True)
df_clues = df_clues.drop(columns=["index"])

df_clues_norm2 = json_normalize(df_clues["PlayerAnswers"])
df_clues_merge = df_clues.merge(df_clues_norm2, left_index=True, right_index=True)
df_clues = df_clues_merge.drop(columns=["PlayerAnswers"])

df_clues = flattenColumn(df_clues, "GameState")
df_clues.reset_index(inplace=True)
df_clues = df_clues.drop(columns=["index"])

df_clues_norm3 = json_normalize(df_clues["GameState"])
df_clues_merge = df_clues.merge(df_clues_norm3, left_index=True, right_index=True)
df_clues = df_clues_merge.drop(columns=["GameState"])

# Release unused dataframes from memory
list_df = [df_clues_norm1, df_clues_norm2, df_clues_norm3, df_clues_merge]
for i in list_df:
    releaseDF(i)

In [37]:
df_clues.head(5)

,GameId,RoundType,HasScoreCorrection,Order_x,Amount_x,Question,CorrectAnswer,Category.Name,Category.Order,AlexComment,IsDailyDouble,QuestionHasMedia,QuestionMediaLink,IsNeverSeen,Order_y,Answer,Wager,Player.PlayerId_x,Player.NickName_x,IsIncorrect,Player.TeamName_x,Player.PlayerId_y,Player.NickName_y,Amount_y,Player.TeamName_y
0,6527,First Round,False,1.0,800.0,"Greek for ""exaggeration"" gives us this device,...",hyperbole,LITERARY TERMS & DEVICES,3,NaN,NaN,NaN,NaN,NaN,1,hyperbole,800.0,13087,James,NaN,NaN,13085,Brad,NaN,NaN
1,6527,First Round,False,1.0,800.0,"Greek for ""exaggeration"" gives us this device,...",hyperbole,LITERARY TERMS & DEVICES,3,NaN,NaN,NaN,NaN,NaN,1,hyperbole,800.0,13087,James,NaN,NaN,13086,Ken,NaN,NaN
2,6527,First Round,False,1.0,800.0,"Greek for ""exaggeration"" gives us this device,...",hyperbole,LITERARY TERMS & DEVICES,3,NaN,NaN,NaN,NaN,NaN,1,hyperbole,800.0,13087,James,NaN,NaN,13087,James,800.0,NaN
3,6527,First Round,False,2.0,1000.0,"102 minus 10 to this Latin bandleader, ""The Ru...",Xavier Cugat,MATH TO ROMAN NUMERALS TO INITIALS TO NAMES,5,"Yes--90, XC.",NaN,NaN,NaN,NaN,1,Xavier Cugat,1000.0,13086,Ken,NaN,NaN,13085,Brad,NaN,NaN
4,6527,First Round,False,2.0,1000.0,"102 minus 10 to this Latin bandleader, ""The Ru...",Xavier Cugat,MATH TO ROMAN NUMERALS TO INITIALS TO NAMES,5,"Yes--90, XC.",NaN,NaN,NaN,NaN,1,Xavier Cugat,1000.0,13086,Ken,NaN,NaN,13086,Ken,1000.0,NaN


In [0]:
df_clues.rename(columns={"Category.Name": "CategoryName",
                         "Category.Order": "CategoryOrder",
                         "Player.PlayerId_x": "PlayerId_x",
                         "Player.NickName_x": "NickName_x",
                         "Player.TeamName_x": "TeamName_x",
                         "Player.PlayerId_y": "PlayerId_y",
                         "Player.NickName_y": "NickName_y",
                         "Player.TeamName_y": "TeamName_y",
                        },
                inplace=True)

In [0]:
# Replace NaN values in IsIncorrect with "False"
df_clues["IsIncorrect"].fillna(False, inplace = True)

# Replace NaN values in IsDailyDouble with "False"
df_clues["IsDailyDouble"].fillna(False, inplace = True)

In [0]:
## Leave NaNs in:
##    Amount_x
##    Order_x
##    AlexComment
##
#df_clues.Order_x.unique()           # NaNs represent questions not played.
#df_clues.Amount_x.unique()          # NaNs represent questions not played.
#df_clues.AlexComment.unique() 
#df_clues.QuestionMediaLink.unique() # NaNs represent questions not played.

In [41]:
df_clues.head(5)

,GameId,RoundType,HasScoreCorrection,Order_x,Amount_x,Question,CorrectAnswer,CategoryName,CategoryOrder,AlexComment,IsDailyDouble,QuestionHasMedia,QuestionMediaLink,IsNeverSeen,Order_y,Answer,Wager,PlayerId_x,NickName_x,IsIncorrect,TeamName_x,PlayerId_y,NickName_y,Amount_y,TeamName_y
0,6527,First Round,False,1.0,800.0,"Greek for ""exaggeration"" gives us this device,...",hyperbole,LITERARY TERMS & DEVICES,3,NaN,False,NaN,NaN,NaN,1,hyperbole,800.0,13087,James,False,NaN,13085,Brad,NaN,NaN
1,6527,First Round,False,1.0,800.0,"Greek for ""exaggeration"" gives us this device,...",hyperbole,LITERARY TERMS & DEVICES,3,NaN,False,NaN,NaN,NaN,1,hyperbole,800.0,13087,James,False,NaN,13086,Ken,NaN,NaN
2,6527,First Round,False,1.0,800.0,"Greek for ""exaggeration"" gives us this device,...",hyperbole,LITERARY TERMS & DEVICES,3,NaN,False,NaN,NaN,NaN,1,hyperbole,800.0,13087,James,False,NaN,13087,James,800.0,NaN
3,6527,First Round,False,2.0,1000.0,"102 minus 10 to this Latin bandleader, ""The Ru...",Xavier Cugat,MATH TO ROMAN NUMERALS TO INITIALS TO NAMES,5,"Yes--90, XC.",False,NaN,NaN,NaN,1,Xavier Cugat,1000.0,13086,Ken,False,NaN,13085,Brad,NaN,NaN
4,6527,First Round,False,2.0,1000.0,"102 minus 10 to this Latin bandleader, ""The Ru...",Xavier Cugat,MATH TO ROMAN NUMERALS TO INITIALS TO NAMES,5,"Yes--90, XC.",False,NaN,NaN,NaN,1,Xavier Cugat,1000.0,13086,Ken,False,NaN,13086,Ken,1000.0,NaN


In [0]:
#download_csv('df_clues.csv')